In [1]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math

In [2]:
def calc_enmo(some_data):
    # Calculate the magnitue by first squaring all of the x, y, and z value, then summing them, and taking the square root.
    mag = ((some_data.applymap(lambda x : x**2)).sum(axis=1)).transform(lambda x : np.sqrt(x))
    # To calculate ENMO we subtract 1 (Gravity) from the vector magnitudes
    enmo = mag.transform(lambda x : x - 1)
    # Finally if we have any ENMO values less than 0 we round them up.
    enmo.loc[enmo.loc[:] < 0] = 0
    
    return mag, enmo

In [3]:
def calc_mad(some_data, device):
    some_data = some_data.dropna()
    all_mad = {}

    time_name = device + " Time"
    some_data[time_name] = some_data[time_name].apply(lambda x: x.replace(microsecond=0))
        
    # Grab the first timestamp from data
    # print(some_data.shape)
    start = some_data.loc[some_data.index[0], time_name]
    # Specify the amount of time to aggregate over
    agg_len = 5
    # Grab end of aggregation period
    end_time = start + timedelta(seconds=agg_len - 1)
    # Calculate the total length of the trial in seconds
    trial_length = (some_data.loc[some_data.index[-1], time_name] - start).total_seconds()
    # Runs the total length of trial divided by the length of time we aggregate over
    # essentialy creates a window of agg_len, and interval of agg_len
    for i in range(int(trial_length//agg_len)):
        # print(end_time)
        # Get agg_len seconds worth of accelerometer readings
        group_s = some_data.loc[(some_data[time_name] >= start) & (some_data[time_name] <= end_time), :]
        # print(group_s)
        # Get the mean X, Y, and Z of those readings
        agg_s = group_s.aggregate(lambda x : np.mean(x))
        # print(mean_s)
        # Calculate the mean accelerometer magnitude
        mag_s = agg_s[4]
        # print(f"{mag_s}")
        # Subtract the mean magnitude from each accelerometer magnitude from each vector magnitude and then take abs
        dif_mean = group_s[device + ' Magnitude'].apply(lambda x : abs(x - mag_s))
        # Caclulate the sum of all the vector mags - mean mags. Then divide by the number of vectors
        # print(dif_mean.sum())
        if device == "Actigraph":
            if dif_mean.shape[0] != 500 :
                print(f"Error {dif_mean.shape[0]} readings")
        mad = (dif_mean.sum()) / dif_mean.shape[0]
        # print(mad)

        # Add each Mad and the corresponding time to a list :
        all_mad[end_time] = mad
        # 
        start = end_time + timedelta(seconds=1)
        end_time = start + timedelta(seconds=agg_len - 1)

    mad_df = pd.Series(data=all_mad)
    return mad_df

In [37]:
# Define path of file
path = "C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\"
trial = input("Which Trial is being processed? ")
input_path = path + "Trial " + trial + "\\apple_aligned.csv"

Which Trial is being processed? 08


In [38]:
# Creates a dictionary that contains the data of each trial, and its keys are each trial number.
temp = pd.read_csv(input_path)
temp["Actigraph Time"] = pd.to_datetime(temp["Actigraph Time"])
temp["Proxy Time"] = pd.to_datetime(temp["Proxy Time"])
acti1, acti2, acti3, acti4, acti5 = temp.groupby("Actigraph ID")
# acti1, acti2, acti3, acti4 = temp.groupby("Actigraph ID")
data = [acti1[1], acti2[1], acti3[1], acti4[1], acti5[1]]
# data = [acti1[1], acti2[1], acti3[1], acti4[1]]
print("Finished") 

Finished


In [39]:
# Calculate Magnitude and ENMO for each device in each trial.
pair_num = 0
for dev_pair in data :# Each trial has 5 device pairs.
    # Calculate magnitude and ENMO of actigraph
    # print(pair_num)
    acti_mag, acti_enmo = calc_enmo( dev_pair.loc[:, ["Actigraph X", "Actigraph Y", "Actigraph Z"]] )
    dev_pair.insert(10, "Actigraph Magnitude", acti_mag)
    dev_pair.insert(11, "Actigraph ENMO", acti_enmo)
    # Calculate magintude and ENMO of proxy
    proxy_mag, proxy_enmo = calc_enmo( dev_pair.loc[:, ["Proxy X", "Proxy Y", "Proxy Z"]] )
    dev_pair.insert(18, "Proxy Magnitude", proxy_mag)
    dev_pair.insert(19, "Proxy ENMO", proxy_enmo)
    dev_pair.loc[(dev_pair["Proxy X"].isna()), ["Proxy Magnitude", "Proxy ENMO"]] = np.nan
    if pair_num == 0 :
        trial_data = dev_pair
    elif pair_num < 3:
        trial_data = pd.concat([trial_data, dev_pair])
    else :
        trial_data = pd.concat([trial_data, dev_pair])
        trial_data.to_csv(path + "Trial " + trial + "//apple_aligned_v2.csv", index=False)
    pair_num += 1

In [40]:
trial_data

,Trial Number,Round Number,Speed,Actigraph ID,Actigraph Time,Order,Include,Actigraph X,Actigraph Y,Actigraph Z,Actigraph Magnitude,Actigraph ENMO,Proxy ID,Proxy Time,Reading #,Proxy X,Proxy Y,Proxy Z,Proxy Magnitude,Proxy ENMO
504000,8,4,3.2 Hz,36,2022-04-01 14:36:15,504001,0,0.461,-0.082,-0.980,1.086115,0.086115,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
504001,8,4,3.2 Hz,36,2022-04-01 14:36:15,504002,0,0.461,-0.227,-1.000,1.124300,0.124300,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
504002,8,4,3.2 Hz,36,2022-04-01 14:36:15,504003,0,0.426,-0.363,-1.004,1.149461,0.149461,V3KQT9D9NR,2022-04-01 14:36:15.016,NaN,0.001465,-0.033203,-0.993164,0.993720,0.000000
504003,8,4,3.2 Hz,36,2022-04-01 14:36:15,504004,0,0.363,-0.449,-0.988,1.144340,0.144340,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
504004,8,4,3.2 Hz,36,2022-04-01 14:36:15,504005,0,0.262,-0.445,-0.992,1.118362,0.118362,V3KQT9D9NR,2022-04-01 14:36:15.036,NaN,0.001221,-0.038086,-1.000000,1.000726,0.000726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839995,8,4,0.6 Hz,113,2022-04-01 14:50:14,839996,0,-0.004,-0.008,-1.043,1.043038,0.043038,GKGGN79P34,2022-04-01 14:50:14.949,NaN,0.015381,-0.005615,-0.993408,0.993543,0.000000
839996,8,4,0.6 Hz,113,2022-04-01 14:50:14,839997,0,-0.004,-0.008,-1.043,1.043038,0.043038,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
839997,8,4,0.6 Hz,113,2022-04-01 14:50:14,839998,0,-0.004,-0.008,-1.043,1.043038,0.043038,GKGGN79P34,2022-04-01 14:50:14.968,NaN,0.012939,-0.003906,-0.996826,0.996918,0.000000
839998,8,4,0.6 Hz,113,2022-04-01 14:50:14,839999,0,-0.004,-0.008,-1.043,1.043038,0.043038,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
def aggregate_data(data, device):
    # Calculate the MAD for device
    device_mad = calc_mad(data.loc[:, [device + " Time", device + " X", device + " Y", device + " Z", 
                                               device + " Magnitude"]], device).rename(device + " MAD")
    # Intialize formula used to calculate RMS
    rms = lambda x : np.sqrt(np.mean(x**2))
    # Intialize functions to be used on data columns
    device_aggs = {"Order" : [np.min], device + " X": [rms], device + " Y": [rms], device + " Z": [rms], device + " Magnitude": [np.max],
                    device + " ENMO": [np.max]}
    # Group the actigraph data by second and perform aggregations
    if device == "Proxy":
        data[device + " Time"] = data[device + " Time"].apply(lambda x : x.replace(microsecond=0))
        device_rms = data.drop(columns=["Reading #"]).dropna().groupby(["Include", device + " Time"]).agg(device_aggs)
        # print(device_rms)
    else:
        device_rms = data.groupby(["Include", device + " Time"]).agg(device_aggs)
        # print(device_rms)
                            
    # print(device_rms)
    # Rename aggregate  columns
    device_rms= device_rms.reset_index()
    device_rms.columns = ["Include", device + " Time", "Order", device + " RMS X", device + " RMS Y", device + " RMS Z", 
                        device + " MAX MAGNITUDE", device + " MAX ENMO"]
    # Selecte middle minutes from eacht trial only
    device_rms = device_rms.loc[device_rms["Include"] == 1].drop(columns=["Include"])
    # print(device_rms)
    # Create a data frame that holds the meta trial data
    acti_meta = data.loc[:,["Trial Number", "Round Number", "Speed", device + " ID", "Order"]]
    #if device == "Proxy":
        #print(device_rms)
        #print(device_mad)
    # Merge the MAD calculation with the aggregated x, y, z, magintude, and ENMO
    device_mad = device_rms.merge(device_mad, how="left", left_on=device + " Time", right_on=device_mad.index)
    # Merge aggregated data with trial meta data
    device_sec = acti_meta.merge(device_mad, how='inner', on="Order")
    # print(device_sec.loc[55:65,:])
    return device_sec

In [42]:
# Calculate MAD for each trial, and also aggregate data to second level.
# Used to keep track of which device pair is being looked at
pair_num = 0
# Resets trial file
trial_final = None
# Iterate through each device pair (5 actigraphs  and 2 proxys per trial)
acti_num = 1
for acti_pair in data :
    print(f"Actigraph Number : {acti_num}")
    # acti_pair["Round Number"] = 5 # Used for extra rounds
    # split the data by proxy ID
    
    [proxy_1, proxy_2] = acti_pair.groupby("Proxy ID")
    # if acti_num == 1 :
        # print(f"Proxy 1: {proxy_1[0]} \nProxy 2: {proxy_2[0]}")
        # print(f"Proxy {acti_pair.iloc[1,6]}")
    acti_num += 1
    # print(proxy_1[1])

    # Aggreagte data and calculate mad for actigraph
    acti_data = aggregate_data(acti_pair.iloc[:acti_pair.shape[0]//2, :], "Actigraph")
    # acti_data = aggregate_data(acti_pair, "Actigraph")
    # print(acti_data)

    # Aggreagte data and calculate mad for proxy 1:
    proxy1_data = aggregate_data(proxy_1[1], "Proxy")
    # proxy1_data = aggregate_data(acti_pair, "Proxy")
    # print(proxy1_data)

    # Aggreagte data and calculate mad for proxy 2:
    proxy2_data = aggregate_data(proxy_2[1], "Proxy")

    # Get rid of duplicate columns
    proxy1_data.drop(columns=["Trial Number", 'Round Number', 'Speed', 'Order'], inplace=True)
    # print(proxy1_data)
    proxy2_data.drop(columns=["Trial Number", 'Round Number', 'Speed', 'Order'], inplace=True)
    # combine into one dataframe
    temp_prox1 = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")

    temp_prox2 = acti_data.merge(proxy2_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")


    if pair_num == 0 :
        trial_final = pd.concat([temp_prox1, temp_prox2])
        # trial_final = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")
    else:
        temp = pd.concat([temp_prox1, temp_prox2])
        # temp = acti_data.merge(proxy1_data, how='inner', left_on="Actigraph Time", right_on="Proxy Time")
        trial_final = pd.concat([trial_final, temp])
        if pair_num == 3 :
            trial_final.to_csv(path + "Trial " + trial + "//apple_rms_v2.csv", index=False)
    pair_num += 1
trial_final

Actigraph Number : 1
Actigraph Number : 2
Actigraph Number : 3
Actigraph Number : 4
Actigraph Number : 5


,Trial Number,Round Number,Speed,Actigraph ID,Order,Actigraph Time,Actigraph RMS X,Actigraph RMS Y,Actigraph RMS Z,Actigraph MAX MAGNITUDE,Actigraph MAX ENMO,Actigraph MAD,Proxy ID,Proxy Time,Proxy RMS X,Proxy RMS Y,Proxy RMS Z,Proxy MAX MAGNITUDE,Proxy MAX ENMO,Proxy MAD
0,8,4,3.2 Hz,36,507001,2022-04-01 14:36:45,0.333257,0.370332,0.999747,1.271325,0.271325,NaN,GKGGN79P34,2022-04-01 14:36:45,0.368725,0.332636,0.996497,1.420176,0.420176,NaN
1,8,4,3.2 Hz,36,507101,2022-04-01 14:36:46,0.326607,0.368713,1.000287,1.292927,0.292927,NaN,GKGGN79P34,2022-04-01 14:36:46,0.387843,0.342131,0.992841,1.407786,0.407786,NaN
2,8,4,3.2 Hz,36,507201,2022-04-01 14:36:47,0.327838,0.370342,1.000706,1.293750,0.293750,NaN,GKGGN79P34,2022-04-01 14:36:47,0.378481,0.328293,0.996873,1.426258,0.426258,NaN
3,8,4,3.2 Hz,36,507301,2022-04-01 14:36:48,0.334626,0.363739,0.998958,1.260998,0.260998,NaN,GKGGN79P34,2022-04-01 14:36:48,0.392229,0.337171,0.997121,1.449574,0.449574,NaN
4,8,4,3.2 Hz,36,507401,2022-04-01 14:36:49,0.328583,0.371228,0.999990,1.290077,0.290077,3.723393e-02,GKGGN79P34,2022-04-01 14:36:49,0.384043,0.337842,0.995815,1.389665,0.389665,0.061571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,8,4,0.6 Hz,113,752501,2022-04-01 14:49:40,0.004000,0.008000,1.043000,1.043038,0.043038,NaN,V3KQT9D9NR,2022-04-01 14:49:40,0.010196,0.035143,0.995211,1.008798,0.008798,NaN
416,8,4,0.6 Hz,113,752601,2022-04-01 14:49:41,0.004000,0.008000,1.043000,1.043038,0.043038,NaN,V3KQT9D9NR,2022-04-01 14:49:41,0.007924,0.028625,0.995497,1.012173,0.012173,NaN
417,8,4,0.6 Hz,113,752701,2022-04-01 14:49:42,0.004000,0.008000,1.043000,1.043038,0.043038,NaN,V3KQT9D9NR,2022-04-01 14:49:42,0.009988,0.033347,0.995322,1.018072,0.018072,NaN
418,8,4,0.6 Hz,113,752801,2022-04-01 14:49:43,0.004000,0.008000,1.043000,1.043038,0.043038,NaN,V3KQT9D9NR,2022-04-01 14:49:43,0.008110,0.030995,0.995260,1.011934,0.011934,NaN


In [ ]:
# Combine all aligned
aligned_files = glob.glob(path + "Trial *//*aligned_V2.csv")
aligned_stack = None
for file in aligned_files:
    if aligned_stack is None:
        aligned_stack = pd.read_csv(file)
    else:
        temp = pd.read_csv(file)
        aligned_stack = pd.concat([aligned_stack, temp])
aligned_stack.to_csv(path + "//apple_aligned_mad_enmo.csv", index=False)

In [190]:
aggregated_files = glob.glob(path + "Trial *//*rms_v2.csv")
aggregated_stack = None
for file in aggregated_files:
    if aggregated_stack is None:
        aggregated_stack = pd.read_csv(file)
    else:
        temp = pd.read_csv(file)
        aggregated_stack = pd.concat([aggregated_stack, temp])
aggregated_stack.to_csv(path + "//apple_aggregated_mad_enmo.csv")

In [189]:
aggregated_files = glob.glob(path + "Trial *//*rms_v2.csv")
aggregated_files

['C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 01\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 02\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 03\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 04\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 05\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 06\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 06e\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 07\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 08\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial 09\\apple_rms_v2.csv',
 'C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data\\Apple\\Trial